## Анализ ошибок и успехов за период

In [1]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
import os
import sys
import shutil  # упаковка в архив
import warnings  # игнорирование предупреждения о формате при импорте из excel

warnings.simplefilter("ignore")

#### Загрузка данных

In [2]:
%%time
# загрузка данных ошибок, успехов и витрин

# пути к файлам
dir_path = os.getcwd().replace("\\", "/")
error_path = '/'.join(dir_path.split('/')[:3]) + '/error'
error_file_list = os.listdir(error_path)
successful_parh = '/'.join(dir_path.split('/')[:3]) + '/succsessful'
successful_file_list = os.listdir(successful_parh)
showcase_parh = '/'.join(dir_path.split('/')[:3]) + '/showcase/'
showcase_file_list = os.listdir(showcase_parh)
result_path = '/'.join(dir_path.split('/')[:3]) + '/result/'
print('созданы пути к файлам')


созданы пути к файлам
CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
# определение типа файлов в целевой папке по расширению первого файла в директории данных ошибок, другие расширения будет нельзя
try:
    extension = error_file_list[0].split(".")[-1]
except Exception:
    print(
        "проверьте содержимое целевой директории фалов ошибок записи - в ней что-то не так"
    )
    sys.exit()
print(f"ожидаемое расширение для ВСЕХ файлов в целевой директории: {extension}")
# создание фреймов успехов и ошибок
df_error = pd.DataFrame()
for error_file in error_file_list:
    error_file_path = error_path + "/" + error_file
    if extension == "xlsx":
        try:
            df_error_day = pd.read_excel(
                error_file_path, sheet_name="1", index_col=None, dtype="str"
            )
        except Exception:
            print(f"ошибка обработки файла {error_file}, файл пропущен")
    elif extension == "csv":
        try:
            df_error_day = df_ass = pd.read_csv(
                error_file_path, sep=";", encoding="utf-8-sig", dtype="str"
            )
        except Exception:
            print(f"ошибка обработки файла {error_file}, файл пропущен")
    else:
        print(f"файл {error_file} имеет неверный тип, файл пропущен")
    df_error = pd.concat([df_error, df_error_day])
    print(f"\t- успешно обработан файл {error_file_path}")
df_error = df_error.reset_index(drop=True).rename(
    columns={
        "Название субъекта РФ": "region_name",
        "Тип записи": "appointment_type",
        "Post_name": "post_name",
    }
)
df_error["create_ts"] = pd.to_datetime(df_error["create_ts"], format="mixed")
# вставка столбца data в таблицу df_error
df_error.insert(
    0,
    "date",
    df_error["create_ts"].apply(
        lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d")
    ),
)
print("данные ошибок записи сформированы")
df_succsessful = pd.DataFrame()
for succsessful_file in successful_file_list:
    succsessful_file_path = successful_parh + "/" + succsessful_file
    if extension == "xlsx":
        try:
            df_succsessful_day = pd.read_excel(
                succsessful_file_path, sheet_name="1", index_col=None, dtype="str"
            )
        except Exception:
            print(f"ошибка обработки файла {succsessful_file}, файл пропущен")
    elif extension == "csv":
        try:
            df_succsessful_day = df_ass = pd.read_csv(
                succsessful_file_path, sep=";", encoding="utf-8-sig", dtype="str"
            )
        except Exception:
            print(f"ошибка обработки файла {succsessful_file}, файл пропущен")
    else:
        print(f"файл {succsessful_file} имеет неверный тип, файл пропущен")
    df_succsessful = pd.concat([df_succsessful, df_succsessful_day])
    print(f"\t- успешно обработан файл {succsessful_file_path}")
df_succsessful = df_succsessful.reset_index(drop=True).rename(
    columns={
        "Дата": "date",
        "Название субъекта РФ": "region_name",
        "Тип записи": "appointment_type",
    }
)
df_succsessful["date"] = df_succsessful["create_ts"].apply(
    lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d")
)
df_succsessful["slot_ts"] = pd.to_datetime(df_succsessful["slot_ts"], format="mixed")
df_succsessful["create_ts"] = pd.to_datetime(
    df_succsessful["create_ts"], format="mixed"
)
print("данные успешных сессий сформированы")

# создание фрейма данных витрин
df_showcase = pd.read_excel(
    showcase_parh + showcase_file_list[0],
    sheet_name="Сессии по датам",
    index_col=None,
    dtype="str",
)
df_showcase["Дата"] = pd.to_datetime(df_showcase["Дата"], format="mixed")
df_showcase = df_showcase.rename(
    columns={
        "Код региона": "region_code",
        "Регион": "region_name",
        "Дата": "date",
        "Шаг": "step",
        "Всего": "total",
        "Успешно": "succsessful",
        "Ошибки ЕПГУ": "epgu_error",
        "Тех ошибки": "tech_error",
        "Орг ошибки": "org_error",
        "ПОДД": "podd",
        "Брошенные": "draft",
    }
)
df_showcase[df_showcase.columns[4:]] = df_showcase[df_showcase.columns[4:]].apply(
    pd.to_numeric, errors="ignore"
)

ожидаемое расширение для ВСЕХ файлов в целевой директории: csv
	- успешно обработан файл c:/work/comparison_error_successful/error/01.09.csv
	- успешно обработан файл c:/work/comparison_error_successful/error/02.09.csv
	- успешно обработан файл c:/work/comparison_error_successful/error/03.09.csv
	- успешно обработан файл c:/work/comparison_error_successful/error/04.09.csv
	- успешно обработан файл c:/work/comparison_error_successful/error/05.09.csv
	- успешно обработан файл c:/work/comparison_error_successful/error/06.09.csv
	- успешно обработан файл c:/work/comparison_error_successful/error/07.09.csv
	- успешно обработан файл c:/work/comparison_error_successful/error/08.09.csv
	- успешно обработан файл c:/work/comparison_error_successful/error/09.09.csv
	- успешно обработан файл c:/work/comparison_error_successful/error/10.09.csv
	- успешно обработан файл c:/work/comparison_error_successful/error/11.09.csv
	- успешно обработан файл c:/work/comparison_error_successful/error/12.09.csv
	

#### Однообразный диапазон дат

In [5]:
date_begin = max(df_error.date.min(), df_succsessful.date.min(), df_showcase.date.min())
date_end = min(df_error.date.max(), df_succsessful.date.max(), df_showcase.date.max())
df_error = df_error[(df_error.date >= date_begin) & (df_error.date <= date_end)].reset_index(drop=True)
df_succsessful = df_succsessful[(df_succsessful.date >= date_begin) & (df_succsessful.date <= date_end)].reset_index(drop=True)
df_showcase = df_showcase[(df_showcase.date >= date_begin) & (df_showcase.date <= date_end)].reset_index(drop=True)

#### Однообразный перечень регионов

In [6]:
region_error = pd.DataFrame(df_error.region_name).drop_duplicates().reset_index(drop=True)
region_error

,region_name
0,Воронежская область
1,Ханты-Мансийский автономный округ - Югра
2,Саратовская область
3,Краснодарский край
4,Мурманская область
...,...
82,Камчатский край
83,Новгородская область
84,Республика Алтай
85,Федеральное медико-биологическое агентство


#### Набор статистических данных для ошибок записи

In [7]:
df_error.head(3)

,date,session_id,region_name,step_id,step_name,error_code,final_state,mo_oid,mo_name,post_name,slot_ts,create_ts,appointment_type,referral_flg
0,2023-09-01,2a9b7325-46dd-46e6-96d6-0e85b525e090,Воронежская область,3,ServiceOrSpecs,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.36.3408.0.241177,"Поликлиника, БУЗ ВО ""Семилукская РБ им. А. В. ...",NaN,NaN,2023-09-01 08:51:04.900000+03:00,3,0
1,2023-09-01,5e5fff05-4519-4078-817b-f58d153b2a47,Воронежская область,3,ServiceOrSpecs,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.36.3408.0.241177,"Поликлиника, БУЗ ВО ""Семилукская РБ им. А. В. ...",NaN,NaN,2023-09-01 14:20:54.440000+03:00,3,0
2,2023-09-01,add734d4-1e7c-4401-ade9-0d001a779d07,Воронежская область,3,ServiceOrSpecs,0,Ошибка отсутсвует,1.2.643.5.1.13.13.12.2.36.3408.0.241177,"Поликлиника, БУЗ ВО ""Семилукская РБ им. А. В. ...",NaN,NaN,2023-09-01 15:20:11.252000+03:00,3,0


In [8]:
# группировка по дате, региогну и событию
df_error_group = df_error.groupby(['date', 'region_name', 'final_state'])['session_id'].count().reset_index().rename(columns={'final_state': 'event', 'session_id': 'value'})
# перечень всех событий
df_error_event = pd.DataFrame(df_error_group.event).drop_duplicates().reset_index(drop=True).rename(columns={'event':'event'})
# перечни регионов и дат
date_list =df_error.date.unique().tolist()
region_list = df_error.region_name.unique().tolist()
# формирование набора статистических данных для ошибок записи
df_error_statistic = pd.DataFrame()
for date in date_list:
    for region in region_list:
        df_loc_day = df_error_group[(df_error_group.region_name == region) & (df_error_group.date == date)]
        df_loc_day = df_loc_day.merge(df_error_event, on='event', how='right')
        df_loc_day['date'] = df_loc_day['date'].fillna(date)
        df_loc_day['region_name'] = df_loc_day['region_name'].fillna(region)
        df_loc_day['value'] = df_loc_day['value'].fillna(0)
        df_error_statistic = pd.concat([df_error_statistic, df_loc_day])

In [9]:
df_error_statistic

,date,region_name,event,value
0,2023-09-01,Воронежская область,Внутренняя ошибка РМИС при создании записи,225.0
1,2023-09-01,Воронежская область,Значение не найдено,966.0
2,2023-09-01,Воронежская область,Ошибка «РМИС не ответил»,4.0
3,2023-09-01,Воронежская область,Ошибка «СМЭВ-timeout»,3.0
4,2023-09-01,Воронежская область,Ошибка валидации данных пациента,0.0
...,...,...,...,...
5,2023-09-17,г. Байконур,Ошибка во входных параметрах при записи на прием,0.0
6,2023-09-17,г. Байконур,Ошибка отсутсвует,0.0
7,2023-09-17,г. Байконур,Внутренняя ошибка концентратора при создании з...,0.0
8,2023-09-17,г. Байконур,Ошибка валидации схемы,0.0


#### Статистика успешных записей через ЕПГУ

In [10]:
df_succsessful.head(3)

,date,session_id,region_name,sp_oid,sp_name,post_name,slot_ts,create_ts,appointment_type
0,2023-09-01,3287002182,Курская область,1.2.643.5.1.13.13.12.2.46.4352.0.10746,"Поликлиника, ОБУЗ ""ЖГБ""",врач-терапевт участковый,2023-09-06 14:44:00+03:00,2023-09-01 00:15:52.953000+03:00,3
1,2023-09-01,3287254930,Краснодарский край,1.2.643.5.1.13.13.12.2.23.10468.0.312122,"Взрослая поликлиника ГБУЗ ""Городская больница ...",врач-офтальмолог,2023-09-15 09:00:00+03:00,2023-09-01 02:09:45.743000+03:00,3
2,2023-09-01,3287063113,Калининградская область,1.2.643.5.1.13.13.12.2.39.3725.0.434689,Поликлиническое отделение №1 Краснопрудная д. ...,врач-терапевт участковый,2023-09-14 16:00:00+03:00,2023-09-01 00:26:40.835000+03:00,3


In [11]:
df_succsessful_statistic = df_succsessful.groupby(['date', 'region_name'])['session_id'].count().reset_index().rename(columns={'session_id': 'value'})
df_succsessful_statistic.insert(2, 'event', 'Успешная запись (ЕПГУ)')
df_succsessful_statistic

,date,region_name,event,value
0,2023-09-01,Алтайский край,Успешная запись (ЕПГУ),2958
1,2023-09-01,Амурская область,Успешная запись (ЕПГУ),315
2,2023-09-01,Архангельская область,Успешная запись (ЕПГУ),287
3,2023-09-01,Астраханская область,Успешная запись (ЕПГУ),32
4,2023-09-01,Белгородская область,Успешная запись (ЕПГУ),1240
...,...,...,...,...
1427,2023-09-17,Ямало-Ненецкий автономный округ,Успешная запись (ЕПГУ),18
1428,2023-09-17,Ярославская область,Успешная запись (ЕПГУ),551
1429,2023-09-17,г. Москва,Успешная запись (ЕПГУ),1326
1430,2023-09-17,г. Санкт-Петербург,Успешная запись (ЕПГУ),9


#### Статистика использования витрин ЕПГУ

In [12]:
df_showcase.head(10)

,region_code,region_name,date,step,total,succsessful,epgu_error,tech_error,org_error,podd,draft
0,30,Астраханская область,2023-09-01,Шаг 1. Идентификация пациента в регионе. Уника...,3460,3421,0,0,39,0,0
1,30,Астраханская область,2023-09-01,Шаг 2. Выбор специальности. Уникальные пользов...,3421,3241,0,0,4,0,176
2,30,Астраханская область,2023-09-01,Шаг 3. Выбор врача и времени. Уникальные польз...,3241,2614,0,0,253,0,374
3,30,Астраханская область,2023-09-01,Шаг 4. Запись. Уникальные пользователи,2614,861,0,9,66,2,1676
4,32,Брянская область,2023-09-01,Шаг 1. Идентификация пациента в регионе. Уника...,8165,7991,0,0,174,0,0
5,32,Брянская область,2023-09-01,Шаг 2. Выбор специальности. Уникальные пользов...,7991,7445,0,0,23,0,523
6,32,Брянская область,2023-09-01,Шаг 3. Выбор врача и времени. Уникальные польз...,7445,5155,0,0,1763,0,527
7,32,Брянская область,2023-09-01,Шаг 4. Запись. Уникальные пользователи,5155,1368,0,83,124,9,3571
8,33,Владимирская область,2023-09-01,Шаг 1. Идентификация пациента в регионе. Уника...,8213,8128,2,22,61,0,0
9,33,Владимирская область,2023-09-01,Шаг 2. Выбор специальности. Уникальные пользов...,8128,7664,0,2,2,0,460


In [13]:
# успешные записи через витрины
df_showcase_succsessuil = df_showcase[
    df_showcase.step == "Шаг 4. Запись. Уникальные пользователи"
][["date", "region_name", "succsessful"]].rename(columns={"succsessful": "value"})
df_showcase_succsessuil.insert(2, "event", "Успешные записи (витрины)")
# количество ошибок через витрины
df_showcase_error = (
    df_showcase.groupby(["date", "region_name"])[
        ["epgu_error", "tech_error", "org_error", "podd", "draft"]
    ]
    .sum()
    .reset_index()
)
# ошибки записи через витрины
df_showcase_error.rename(
    columns={
        "epgu_error": "Ошибки ЕПГУ (витрины)",
        "tech_error": "Технические ошибки (витрины)",
        "org_error": "Организационные ошибки (витрины)",
        "podd": "ПОДД (витрины)",
        "draft": "Брошеные (витрины)",
    },
    inplace=True,
)
# преобразование в "длинную" таблицу
df_showcase_error = pd.melt(
    df_showcase_error,
    id_vars=["date", "region_name"],
    value_vars=[
        "Ошибки ЕПГУ (витрины)",
        "Технические ошибки (витрины)",
        "Организационные ошибки (витрины)",
        "ПОДД (витрины)",
        "Брошеные (витрины)",
    ],
)
df_showcase_error.rename(columns={"variable": "event"}, inplace=True)
df_showcase_statistic = pd.concat(
    [df_showcase_succsessuil, df_showcase_error]
).sort_values(by=["date", "region_name", "event"]).reset_index(drop=True)
df_showcase_statistic

,date,region_name,event,value
0,2023-09-01,Астраханская область,Брошеные (витрины),2226
1,2023-09-01,Астраханская область,Организационные ошибки (витрины),362
2,2023-09-01,Астраханская область,Ошибки ЕПГУ (витрины),0
3,2023-09-01,Астраханская область,ПОДД (витрины),2
4,2023-09-01,Астраханская область,Технические ошибки (витрины),9
...,...,...,...,...
2869,2023-09-17,Ямало-Ненецкий автономный округ,Организационные ошибки (витрины),299
2870,2023-09-17,Ямало-Ненецкий автономный округ,Ошибки ЕПГУ (витрины),0
2871,2023-09-17,Ямало-Ненецкий автономный округ,ПОДД (витрины),0
2872,2023-09-17,Ямало-Ненецкий автономный округ,Технические ошибки (витрины),11


#### Профиль региона

In [14]:
df_test = pd.concat([df_error_statistic, df_showcase_statistic]).sort_values(by=["region_name"])
df_test.groupby('event').count()

,date,region_name,value
event,,,
Брошеные (витрины),479,479,479
Внутренняя ошибка РМИС при создании записи,1479,1479,1479
Внутренняя ошибка концентратора при создании записи,1479,1479,1479
Значение не найдено,1479,1479,1479
Организационные ошибки (витрины),479,479,479
Ошибка «РМИС не ответил»,1479,1479,1479
Ошибка «СМЭВ-timeout»,1479,1479,1479
Ошибка валидации данных пациента,1479,1479,1479
Ошибка валидации схемы,1479,1479,1479


In [15]:
df_statistic = pd.concat([df_error_statistic, df_succsessful_statistic]).sort_values(by=["date", "region_name"])
df_statistic = pd.concat([df_statistic, df_showcase_statistic]).sort_values(by=["date", "region_name"])

In [16]:
df_statistic[df_statistic.region_name == 'Алтайский край'].head(30)

,date,region_name,event,value
0,2023-09-01,Алтайский край,Внутренняя ошибка РМИС при создании записи,25.0
1,2023-09-01,Алтайский край,Значение не найдено,103.0
2,2023-09-01,Алтайский край,Ошибка «РМИС не ответил»,38.0
3,2023-09-01,Алтайский край,Ошибка «СМЭВ-timeout»,1.0
4,2023-09-01,Алтайский край,Ошибка валидации данных пациента,88.0
5,2023-09-01,Алтайский край,Ошибка во входных параметрах при записи на прием,532.0
6,2023-09-01,Алтайский край,Ошибка отсутсвует,12503.0
7,2023-09-01,Алтайский край,Внутренняя ошибка концентратора при создании з...,0.0
8,2023-09-01,Алтайский край,Ошибка валидации схемы,0.0
9,2023-09-01,Алтайский край,Сервис МИС приостановлен,0.0


______________________________________

In [17]:
for date in date_list:
    df_showcase_test = df_showcase[df_showcase['date'] == date]
    break
df_showcase_test.head(30)

,region_code,region_name,date,step,total,succsessful,epgu_error,tech_error,org_error,podd,draft
0,30,Астраханская область,2023-09-01,Шаг 1. Идентификация пациента в регионе. Уника...,3460,3421,0,0,39,0,0
1,30,Астраханская область,2023-09-01,Шаг 2. Выбор специальности. Уникальные пользов...,3421,3241,0,0,4,0,176
2,30,Астраханская область,2023-09-01,Шаг 3. Выбор врача и времени. Уникальные польз...,3241,2614,0,0,253,0,374
3,30,Астраханская область,2023-09-01,Шаг 4. Запись. Уникальные пользователи,2614,861,0,9,66,2,1676
4,32,Брянская область,2023-09-01,Шаг 1. Идентификация пациента в регионе. Уника...,8165,7991,0,0,174,0,0
5,32,Брянская область,2023-09-01,Шаг 2. Выбор специальности. Уникальные пользов...,7991,7445,0,0,23,0,523
6,32,Брянская область,2023-09-01,Шаг 3. Выбор врача и времени. Уникальные польз...,7445,5155,0,0,1763,0,527
7,32,Брянская область,2023-09-01,Шаг 4. Запись. Уникальные пользователи,5155,1368,0,83,124,9,3571
8,33,Владимирская область,2023-09-01,Шаг 1. Идентификация пациента в регионе. Уника...,8213,8128,2,22,61,0,0
9,33,Владимирская область,2023-09-01,Шаг 2. Выбор специальности. Уникальные пользов...,8128,7664,0,2,2,0,460


In [18]:
# min(df_error.create_ts)
# timestamp_string = "2023-07-21 15:30:45"
# format_string = "%Y-%m-%d %H:%M:%S"
# datetime_object = datetime.strptime(str(min(df_error.create_ts))[:10], '%Y-%m-%d')
datetime.strptime(str(min(df_error.create_ts))[:10], '%Y-%m-%d').date()

datetime.date(2023, 9, 1)

### Количество пропусков

In [19]:
pd.DataFrame(df_error.isnull().sum()).reset_index().rename(columns={'index': 'col', 0: 'count'})

,col,count
0,date,0
1,session_id,0
2,region_name,0
3,step_id,0
4,step_name,0
5,error_code,0
6,final_state,0
7,mo_oid,2657067
8,mo_name,2618447
9,post_name,3766327


In [20]:
pd.DataFrame(df_succsessful.isnull().sum()).reset_index().rename(columns={'index': 'col', 0: 'count'})

,col,count
0,date,0
1,session_id,0
2,region_name,0
3,sp_oid,30544
4,sp_name,0
5,post_name,0
6,slot_ts,0
7,create_ts,0
8,appointment_type,0


In [21]:
pd.DataFrame(df_showcase.isnull().sum()).reset_index().rename(columns={'index': 'col', 0: 'count'})

,col,count
0,region_code,0
1,region_name,0
2,date,0
3,step,0
4,total,0
5,succsessful,0
6,epgu_error,0
7,tech_error,0
8,org_error,0
9,podd,0


In [ ]:
source_path = dir_path + "/source"
save_path = dir_path + "/error_log/"
file_list = os.listdir(source_path)

In [2]:
%%time

import pandas as pd
import os

dir_path = os.getcwd()
dir_ = os.listdir(dir_path.replace("\\", "/"))
list_files = []
for element in dir_:
    try:
        if element.split(".")[-1] == "csv" and element.split(".")[0] != "result":
            list_files.append(element)
    except Exception:
        pass
    try:
        if element.split(".")[-1] == "xlsx":
            part_sh = dir_path.replace("\\", "/") + "/" + element
    except Exception:
        pass
try:
    if os.path.getsize(
        dir_path.replace("\\", "/") + "/" + list_files[0]
    ) > os.path.getsize(dir_path.replace("\\", "/") + "/" + list_files[1]):
        path_error = dir_path.replace("\\", "/") + "/" + list_files[0]
        path_suc = dir_path.replace("\\", "/") + "/" + list_files[1]
    else:
        path_error = dir_path.replace("\\", "/") + "/" + list_files[1]
        path_suc = dir_path.replace("\\", "/") + "/" + list_files[0]
except Exception:
    print("Что-то пошло не так. Будте внимательнее и попробуйте ещё раз.")

path_result = dir_path.replace("\\", "/") + "/result.csv"

df_test = pd.read_csv(path_error, sep=";", dtype=object, header=None)
# замена пустого значения в столбце региона на... непустое значение
df_test[1] = df_test[1].fillna("Регион не определён")
df_check = df_test.copy() # для опытов
df_check_0 = df_test.copy() # для опытов
error_type_list = [
    "Внутренняя ошибка РМИС при создании записи",
    "Ошибка «РМИС не ответил»",
    "Ошибка валидации данных пациента",
    "Ошибка валидации схемы",
    "Сервис МИС приостановлен",
]
error_type_list_fer = [
    "Внутренняя ошибка концентратора при создании записи",
    "Ошибка «СМЭВ-timeout»",
    "Ошибка во входных параметрах при записи на прием",
]
df_tech_error = df_test.loc[(df_test[5].isin(error_type_list)) & (df_test[11] == "3")]
df_tech_error = pd.DataFrame(df_tech_error.groupby(1)[5].count()).reset_index()
df_org_error = df_test.loc[(df_test[5] == "Значение не найдено") & (df_test[11] == "3")]
df_org_error = pd.DataFrame(df_org_error.groupby(1)[5].count()).reset_index()
df_regions = (
    pd.DataFrame(df_test[1]).drop_duplicates().drop(index=0).reset_index(drop=True)
)
df_fer_error = df_test.loc[
    (df_test[5].isin(error_type_list_fer)) & (df_test[11] == "3")
]

# df_fer_error_test = df_fer_error.copy() # для проверки ошибок ФЕР с нулевым регионом
df_fer_error = pd.DataFrame(df_fer_error.groupby(1)[5].count()).reset_index()

df_errors = (
    df_regions.merge(df_tech_error, left_on=1, right_on=1, how="left")
    .merge(df_org_error, left_on=1, right_on=1, how="left")
    .merge(df_fer_error, left_on=1, right_on=1, how="left")
)
df_errors = df_errors.rename(
    columns={
        1: "region_name",
        "5_x": "tech_errors",
        "5_y": "org_errors",
        5: "FER_errors",
    }
)
df_errors = df_errors.fillna(0)
df_errors[["tech_errors", "org_errors", "FER_errors"]] = df_errors[
    ["tech_errors", "org_errors", "FER_errors"]
].astype("int")
df_test = pd.read_csv(path_suc, sep=";", dtype=object, header=None)
df_suc_test = df_test.copy() # для опытов 2
# замена пустого значения в столбце региона на... непустое значение
df_test[2] = df_test[2].fillna("Регион не определён")

df_suc = pd.DataFrame(
    df_test.loc[df_test[8] == "3"].groupby(2).count()[0]
).reset_index()
df_full = (
    df_errors.merge(df_suc, left_on="region_name", right_on=2, how="left")
    .rename(columns={0: "success"})[
        ["region_name", "success", "org_errors", "FER_errors", "tech_errors"]
    ]
    .fillna(0)
)
df_full["success"] = df_full["success"].astype("int")

df_full["success_ratio(%)"] = round(
    df_full["success"]
    * 100
    / (df_full["success"] + df_full["org_errors"] + df_full["tech_errors"]),
    1,
).fillna(100.0)
df_full["org_errors_ratio(%)"] = round(
    df_full["org_errors"]
    * 100
    / (df_full["success"] + df_full["org_errors"] + df_full["tech_errors"]),
    1,
).fillna(0.00)
df_full["tech_errors_ratio(%)"] = round(
    df_full["tech_errors"]
    * 100
    / (df_full["success"] + df_full["org_errors"] + df_full["tech_errors"]),
    1,
).fillna(0.00)
df_full = df_full[
    [
        "region_name",
        "success",
        "success_ratio(%)",
        "org_errors",
        "org_errors_ratio(%)",
        "tech_errors",
        "tech_errors_ratio(%)",
        "FER_errors",
    ]
]

df_full = df_full.sort_values("success_ratio(%)", ascending=False).reset_index(
    drop=True
)

sum_row = {
    col: df_full[col].sum()
    for col in ["success", "org_errors", "FER_errors", "tech_errors"]
}
sum_df = pd.DataFrame(sum_row, index=["Total"])
sum_df.insert(0, "region_name", "TOTAL")
sum_df = sum_df.reset_index(drop=True)
df_full = pd.concat([df_full, sum_df], ignore_index=True).fillna("")

df_full.to_csv(path_result, sep=";", encoding="utf-8-sig", index=False)
print(
    "\n\n\t\t\tКоличество (доля) успешных записей и ошибок по субъектам (ошибки ФЭР приведены для справки, в расчёте долей не применяются)\n"
)
print(df_full.to_string(index=False))

# проверка на пустой регион
df_check = df_check.rename(columns=df_check.iloc[0]).drop(index=0)
df_check_actual = df_check[
    ((df_check["final_state"].isin(error_type_list))
    | (df_check["final_state"].isin(error_type_list_fer))
    | (df_check["final_state"] == "Значение не найдено"))
    & ((df_check["Тип записи"] == "3"))
]
df_check_nanregion = df_check_actual[df_check_actual["Название субъекта РФ"] == "Регион не определён"]
if df_check_nanregion.empty:
    print("Пустых регионов нет")
else:
    print("\n\n\t\t\tПустые регионы:\n")
    print(df_check_nanregion[
    [
        "session_id",
        "step_id",
        "step_name",
        "error_code",
        "final_state",
    ]
].to_string(index=True))
print("\n\n\t\t\tВсё\n")



			Количество (доля) успешных записей и ошибок по субъектам (ошибки ФЭР приведены для справки, в расчёте долей не применяются)

                             region_name  success success_ratio(%)  org_errors org_errors_ratio(%)  tech_errors tech_errors_ratio(%)  FER_errors
                   Волгоградская область     2949             91.6         108                 3.4          164                  5.1         482
                         Республика Тыва      830             89.4          32                 3.4           66                  7.1          56
                    Чеченская Республика     2156             89.1          66                 2.7          198                  8.2          81
                    Чувашская Республика     3262             89.1         122                 3.3          278                  7.6         542
         Карачаево-Черкесская Республика      187             86.2           7                 3.2           23                 10.6          25


In [2]:
path_error

'c:/work/new_bot/script/09.08.csv'

In [4]:
# part_sh
df_sh = pd.read_excel(
    part_sh, sheet_name="Сессии по датам", index_col=None, dtype="str"
)
df_sh = df_sh.apply(pd.to_numeric, errors="ignore")
df_sh["Дата"] = pd.to_datetime(df_sh["Дата"], format="%Y-%m-%d %H:%M:%S")
df_sh_group = df_sh.loc[df_sh['Дата'] == df_sh['Дата'].max()][[df_sh.columns[1], *list(df_sh.columns[4:])]].groupby('Регион').sum().reset_index()

In [5]:
df_sh_group

,Регион,Всего,Успешно,Ошибки ЕПГУ,Тех ошибки,Орг ошибки,ПОДД,Брошенные
0,Астраханская область,14898,11874,0,96,402,19,2507
1,Брянская область,37157,28173,1,100,2717,59,6107
2,Владимирская область,32055,24629,1,6,1717,69,5633
3,Вологодская область,18667,14178,0,12,1496,0,2981
4,Иркутская область,20757,15423,0,46,976,1,4311
5,Кабардино-Балкарская Республика,10578,8832,0,2,124,0,1620
6,Калужская область,24949,19215,0,25,1550,1,4158
7,Кировская область,24643,18840,0,1,1515,0,4287
8,Курганская область,3478,2559,0,56,247,0,616
9,Липецкая область,25932,18839,0,164,2034,579,4316


In [41]:
df_allfull = df_full.merge(df_sh_group, left_on='region_name', right_on='Регион', how='left')

In [43]:
df_allfull
print(df_allfull.to_string(index=False))

                               region_name  success success_ratio(%)  org_errors org_errors_ratio(%)  tech_errors tech_errors_ratio(%)  FER_errors                                   Регион    Всего  Успешно  Ошибки ЕПГУ  Тех ошибки  Орг ошибки  ПОДД  Брошенные
Федеральное медико-биологическое агентство        0            100.0           0                 0.0            0                  0.0           0                                      NaN      NaN      NaN          NaN         NaN         NaN   NaN        NaN
                           Республика Тыва     1199             89.9          64                 4.8           70                  5.3         179                                      NaN      NaN      NaN          NaN         NaN         NaN   NaN        NaN
                     Волгоградская область     2110             87.6         127                 5.3          173                  7.2         698                                      NaN      NaN      NaN          NaN  